## **Car Insurance Analysis**

Hey,
this project is a bonus material from DataCamp course. In this project, I will fundamentally answer a single question: **which variable is the best predictor of whether a costumer will put in a claim on their car insurance?**
According to the project's tale, this variable will be the single feature in a machine learning model to try to correctly predict this outcome. 

*DataCamp background for the project*

Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

## Importing libs and defining functions

To perform this analysis, I will use the pandas and numpy libraries to clean the data and the ```logit``` function from statsmodels to perform the logistic regression. Also, as the exercise didn't grant a tidy data frame, I'll explore the data and check its columns.

### Why logit? What's Logit anyway? 

The dependent variable, also called *response variable*, to be studied is known as binary because it assumes ``0`` or ``1``. When it's ``1``, the client made a claim on their car insurance and ``0`` otherwise. So, what I am measuring here it's the probability change due to the change in one of the independent variables. Logit models are regressions using the logistic function as the underlying probability function. It grants that the probability will be between 0 and 1. That's why I'm importing ``logit`` instead of the ```ols``` function.

In [1]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Functions to assess the cleaniness of the data
def assert_col(df, col, cat):
    """Assert if a column from a data frame has only the indicate number for categories.
    
    Args:
        df (PandasDataFrame): a PandasDataframe.
        col (str): the df's column to be verified.
        cat (int): the expected number of categories.
        
    Returns:
        print: a message to indicate if the column is clean, i.e., a codified categorical variable or not."""
    try:
        assert len(df[col].unique()) == cat
        print(f'the {col} column match its description.')
    except AssertionError as e:
        print(f'the {col} column doesn\'t match its description.')
        print(e)

def assert_cat(df, col):
    """Assert if the column is categorical. If not categorical, convert to the categorical type.
    
    Args:
        df (PandasDataFrame): a PandasDataframe.
        col (str): the df's column to be verified.
        
    Returns:
        PandasSeries: returns the column in Series format converted to categorical if necessary. 
        It also displays a message, indicating if the astype method was necessary"""
    col_type = str(df[col].dtype)
    if col_type != 'category':
        df[col] = df[col].astype('category')
        print(f'the {col} column is not category. astype method applied.')
        return df[col]
    else:
        print(f'the {col} column is a category. Returning the column.')
        return df[col]

# Function to check the logit model accuracy
def accuracy(logit_model):
    """Return the accuracy of the logit model. A logit's model accuracy measure the proportion 
    of correct predictions of the dataset. It is the ratio between the correct predictions and all
    the model predictions.
    
    Args:
        logit_model (logit model): a fitted logit model.
        
    Returns:
        int: the accuracy of the logit model"""
    conf_matrix = logit_model.pred_table()
    acc = (conf_matrix[0,0] + conf_matrix[1,1])/(conf_matrix[0,0] + conf_matrix[1,1] + conf_matrix[1,0] + conf_matrix[0,1])
    return acc

## Reading and Exploring the Data Frame

In [2]:
df = pd.read_csv('car_insurance.csv')

In [3]:
# df's first five rows
print(df.head())

       id  age  gender driving_experience    education         income  \
0  569520    3       0               0-9y  high school    upper class   
1  750365    0       1               0-9y         none        poverty   
2  199901    0       0               0-9y  high school  working class   
3  478866    0       1               0-9y   university  working class   
4  731664    1       1             10-19y         none  working class   

   credit_score  vehicle_ownership vehicle_year  married  children  \
0      0.629027                1.0   after 2015      0.0       1.0   
1      0.357757                0.0  before 2015      0.0       0.0   
2      0.493146                1.0  before 2015      0.0       0.0   
3      0.206013                1.0  before 2015      0.0       1.0   
4      0.388366                1.0  before 2015      0.0       0.0   

   postal_code  annual_mileage vehicle_type  speeding_violations  duis  \
0        10238         12000.0        sedan                    0  

In [4]:
# Overall information of the dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

## Cleaning the dataset

Most of the independent variables are categorical. However, they are presented as either integers or strings in the dataset. Both settings can be used to be categories. The problem is that neither integers nor strings serves us as category. To ensure these columns are treated as categories, we have to verify two conditions: the number of categories match its description in *the dataset* section and its `dtype` is category. In the first condition, I'm assuming that any error or mistype is bound to generate more categories than the ones already placed in its description. The second condition **warrants** the column will be treated as a category.

As for the numerical ones, `credit_score` and `annual_mileage` presented null values. Usually, when the dataset have at least one of its row with more than 5% with null values, we'll impute the values with statistical measures (mean or median). In both cases, more than 5% of the column is missing, so I'll impute either the mean or the median, depending on how far the mean is from the median.

### Cleaning the categories

I will use the `assert_col` function to ensure that the column has the number of the categories of its description and `assert_cat` to verify if its dtype is a category or not (we know it's not, so all will be converted).

In [5]:
# Separate the cat columns in three lists. Based on the dataset section.
cat_4_col = list(df[['age', 'driving_experience','income']].columns)

cat_2_col = list(df[['gender','vehicle_ownership', 'vehicle_year', 'married', 'vehicle_type']].columns)

for col in cat_2_col+cat_4_col+['education']:
    if col in cat_2_col:
        assert_col(df, col, cat = 2)
        df[col] = assert_cat(df, col)
    elif col in cat_4_col:
        assert_col(df, col, cat = 4)
        df[col] = assert_cat(df, col)
    else:
        assert_col(df, col, cat = 3)
        df[col] = assert_cat(df, col)

the gender column match its description.
the gender column is not category. astype method applied.
the vehicle_ownership column match its description.
the vehicle_ownership column is not category. astype method applied.
the vehicle_year column match its description.
the vehicle_year column is not category. astype method applied.
the married column match its description.
the married column is not category. astype method applied.
the vehicle_type column match its description.
the vehicle_type column is not category. astype method applied.
the age column match its description.
the age column is not category. astype method applied.
the driving_experience column match its description.
the driving_experience column is not category. astype method applied.
the income column match its description.
the income column is not category. astype method applied.
the education column match its description.
the education column is not category. astype method applied.


### Cleaning the numerical columns

We'll take the mean and median to check if they are similar for each column. 

In [6]:
# credit_score and annual mileage has NAs - more than 9% of the data frame
# substitute NAs for the mean or median

print(df[['credit_score', 'annual_mileage']].agg(['mean', 'median']))

        credit_score  annual_mileage
mean        0.515813    11697.003207
median      0.525033    12000.000000


As the mean and median of ``credit_score`` and ``annual_mileage`` columns are both similar, I'll use mean as the substitute for NAs in both cases.

In [7]:
# Replacing NAs in credit_score and annual_mileage
mean_credit = df['credit_score'].mean()
mean_mileage = df['annual_mileage'].mean()

df = df.fillna({'credit_score': mean_credit,
                'annual_mileage': mean_mileage})

### ``postal_code`` column case
Postal codes are a category but it is not displayed as such in the dataset table. In fact, there are only four postal codes repeated throughout the dataset. So we have to convert them into category as well.

In [8]:
# Checking the last column to be verified
print(df['postal_code'].head())
print(df['postal_code'].unique())

0    10238
1    10238
2    10238
3    32765
4    32765
Name: postal_code, dtype: int64
[10238 32765 92101 21217]


In [9]:
# Postal code are repetitive and are strings, so they are a category
df['postal_code'] = df['postal_code'].astype('category')

### The final dataset

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   id                   10000 non-null  int64   
 1   age                  10000 non-null  category
 2   gender               10000 non-null  category
 3   driving_experience   10000 non-null  category
 4   education            10000 non-null  category
 5   income               10000 non-null  category
 6   credit_score         10000 non-null  float64 
 7   vehicle_ownership    10000 non-null  category
 8   vehicle_year         10000 non-null  category
 9   married              10000 non-null  category
 10  children             10000 non-null  float64 
 11  postal_code          10000 non-null  category
 12  annual_mileage       10000 non-null  float64 
 13  vehicle_type         10000 non-null  category
 14  speeding_violations  10000 non-null  int64   
 15  duis                

## Applying the logit models

Now, we have a fully tidy data frame that can be modelled. Mind that the question to be answer here is: **which variable is the best predictor of whether a costumer will put in a claim on their car insurance?**. As this will be presented as a *single feature* in the machine learning model, we have to analyze one by one its effect on the outcome. In other words, we have to run 16 logit models and calculate its accuracy. To avoid repetition, I'll use list comprehension and the `accuracy` function I created to measure the accuracy of each feature efficiently.

In [11]:
# Scanning all the features to search the best based on accuracy

# listing all the features
cols_df = list(df.columns)
cols_df.remove('outcome')
cols_df.remove('id')

# list with all the logit models
logits = [logit(f'outcome ~ {col}', data = df).fit() for col in cols_df]

# list with the logit models' accuracies
accs = [accuracy(mdl) for mdl in logits]

# Sorting by accuracy to se the best feature by the highest accuracy
features = pd.DataFrame({'feature' : cols_df,
                         'accuracy' : accs})\
    .sort_values('accuracy', ascending=False)

print(features) # we have a winner.

Optimization terminated successfully.
         Current function value: 0.506484
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  

C:\Users\Paulo Vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


The following step was just for the project that required a separated data frame with the best feature.

In [12]:
# Best feature

best_feature_df = pd.DataFrame({'best_feature': [features.iloc[0, 0]],
                             'best_accuracy': [features.iloc[0, 1]]})

best_feature = best_feature_df['best_feature'][0]
best_accuracy = round(best_feature_df['best_accuracy'][0]*100, 1)

print(f'the best feature is {best_feature} with a accuracy of {best_accuracy}%')

the best feature is driving_experience with a accuracy of 77.7%


## Conclusions

This project was meant to select the most accurate single feature in order to make a simple machine learning model to predict if the customer will or will not put a claim on the insurance. To do so, I cleaned the data and then regress the logit of ``outcome`` over each independent variable individually. Based on the accuracy measure alone, the driver's experience proved to be the best feature for a ML model, with an accuracy of 78%.